In [ ]:
# This will import the autoreload extension
%load_ext autoreload

# This will set autoreload to reload all modules (excluding those that can't be reloaded)
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from SARLens.processor.focus import coarseRDA
from SARLens.utils.io import load, plot_with_cdf, plot_with_dask
import torch
import torch.nn.functional as F  

# setting device:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def multiply(a, b):
    """
    Multiplies two arrays a and b using the specified backend.
    """
    return a * b

# Metadata checker:

In [ ]:
# from SARLens.processor.decode import header_extractor
# inputfile = '/Users/robertodelprete/Desktop/AutoFocusNet/Data/DATA/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139.dat'
# meta_rich = header_extractor(inputfile, 'richa')
# meta_s1isp = header_extractor(inputfile, 's1isp')
# meta_rich.to_pickle('meta_rich.pkl')
# meta_s1isp.to_pickle('meta_s1isp.pkl')
# richa_to_s1isp = {
#     "Packet Version Number": "packet_version_number",
#     "Packet Type": "packet_type",
#     "Secondary Header Flag": "secondary_header_flag",
#     "PID": "pid",
#     "PCAT": "pcat",
#     "Sequence Flags": "sequence_flags",
#     "Packet Sequence Count": "packet_sequence_count",
#     "Packet Data Length": "packet_data_length",
#     "Coarse Time": "coarse_time",
#     "Fine Time": "fine_time",
#     "Sync": "sync_marker",
#     "Data Take ID": "data_take_id",
#     "ECC Number": "ecc_num",
#     "Test Mode": "test_mode",
#     "Rx Channel ID": "rx_channel_id",
#     "Instrument Configuration ID": "instrument_configuration_id",
#     "Sub-commutated Ancilliary Data Word Index": "data_word_index",
#     "Space Packet Count": "space_packet_count",
#     "PRI Count": "pri_count",
#     "Error Flag": "error_flag",
#     "BAQ Mode": "baq_mode",
#     "BAQ Block Length": "baq_block_length",
#     "Range Decimation": "range_decimation",
#     "Rx Gain": "rx_gain",
#     "Tx Ramp Rate": "tx_ramp_rate",
#     "Tx Pulse Start Frequency": "tx_pulse_start_freq",
#     "Tx Pulse Length": "tx_pulse_length",
#     "Rank": "rank",
#     "PRI": "pri",
#     "SWST": "swst",
#     "SWL": "swl",
#     "SAS SSB Flag": "ssb_flag",
#     "Polarisation": "polarization",
#     "Temperature Compensation": "temperature_compensation",
#     "Calibration Mode": "cal_mode",
#     "Tx Pulse Number": "tx_pulse_number",
#     "Signal Type": "signal_type",
#     "Swap Flag": "swap",
#     "Swath Number": "swath_number",
#     "Number of Quads": "number_of_quads"
# }

# VERBOSE = False

# idx = 129

# row_rich = meta_rich.iloc[idx]
# row_s1isp = meta_s1isp.iloc[idx]

# keys = list(row_rich.keys())

# non_equal_elements = []

# for key in keys:
#     try: 
#         elem_rich = row_rich[key]
#         elem_s1isp = row_s1isp[richa_to_s1isp[key]]
        
#         if VERBOSE:
#             print(key, elem_rich, float(elem_s1isp), elem_rich == float(elem_s1isp))
        
#         if not elem_rich == float(elem_s1isp):
#             non_equal_elements.append((key, float(elem_rich), float(elem_s1isp)))

#     except:
#         print('Missing key:', key)
#         continue

# for elem in non_equal_elements:
#     print(elem)

# Focusing:

#### Decoding

In [ ]:
# input_file = '/Data_large/marine/PythonProjects/AutoFocusNet/Data/SANPAOLO/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139.dat'
# output_folder = '/Data_large/marine/PythonProjects/AutoFocusNet/Data/Decoded/SANPAOLO'
# !python -m SARLens.processor.decode -i {input_file} -o {output_folder}

### Focusing:

In [ ]:
data_path = '/Data_large/marine/PythonProjects/AutoFocusNet/Data/Decoded/SANPAOLO'

radar_data = load(f'{data_path}/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_pkt_0.pkl')
metadata = load(f'{data_path}/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_pkt_0_metadata.pkl')
ephemeris = load(f'{data_path}/s1a-s3-raw-s-hh-20240524t213606-20240524t213631-054018-069139_ephemeris.pkl')

# init
radar_data = torch.from_numpy(radar_data).to("cpu")

RadarProcessor = coarseRDA(
    raw_data={'echo':radar_data[:,:], 'ephemeris':ephemeris, 'metadata':metadata}, 
    verbose=False, 
    backend='torch'
)

# Perform operations on padded x
H, original_W = RadarProcessor.radar_data.shape
print('Original shape:', H, original_W)

#### Padding with torch functional:

In [ ]:
# desired_width = 128

# # Apply zero padding
# pad_width = desired_width 
# RadarProcessor.radar_data = F.pad(RadarProcessor.radar_data, (pad_width // 2, pad_width // 2, 0, 0), 'constant', 0)

# # Perform operations on padded x
# H, W = RadarProcessor.radar_data.shape
# print('New shape:', H, W)

In [ ]:
H, W = RadarProcessor.radar_data.shape
print('Shape:', H, W)

W_PAD = 2548

RadarProcessor.fft2D(w_pad=W_PAD)

# RG compression
RadarProcessor.radar_data = multiply(RadarProcessor.radar_data, 
                                RadarProcessor.get_range_filter(pad_W=W_PAD))

# Remove padding
start_index = W_PAD // 2
end_index = start_index + original_W
RadarProcessor.radar_data = RadarProcessor.radar_data[:, start_index:end_index]

############ RCMC
RadarProcessor.radar_data = multiply(RadarProcessor.radar_data, 
                                RadarProcessor.get_RCMC())
# IFFT Range 
RadarProcessor.ifft_rg()
# Az Compression
RadarProcessor.radar_data = multiply(RadarProcessor.radar_data, 
                            RadarProcessor.get_azimuth_filter())
# To Rg x Az:
RadarProcessor.ifft_az()

In [ ]:
plot_with_cdf(RadarProcessor.radar_data, (14,14))